In [2]:
from data_preporation_field_scale import field_scale
import geopandas as gpd
import geemap
import ee
import datetime

%load_ext autoreload
%autoreload 2

In [3]:
region_geometry = gpd.read_file('kursk\kursk.geojson')
ROIs = gpd.read_file('kursk\kursk.geojson')
start = '2020-10-15'
finish = '2021-10-15'
import geemap

In [4]:
for_NDTI = field_scale(start , finish , ROIs , region_geometry)
for_minNDTI = field_scale("2021-07-15" , finish , ROIs , region_geometry)

In [5]:
for_NDTI.get_collection()

In [6]:
date_list = for_NDTI.result_collection.aggregate_array('Date').getInfo()
Date_list = [(DL["value"]) for DL in date_list]

def get_time_lst(time_lst):
            unique_dates = []
            for i in time_lst:
                    unique_date_str = str(i)[:10]
                    unique_date = int(unique_date_str)
                    unique_dates.append(datetime.datetime.fromtimestamp(unique_date).strftime("%Y-%m-%d"))
            dates = list(set(unique_dates))
            dates.sort()

            return(dates)

Date_list = get_time_lst(Date_list)
for date in Date_list:
        img = for_NDTI.result_collection.filterMetadata('Date', 'equals', ee.Date(date)).first()
        img = img.select(['B4','B3','B2','B8','B11','B12','NDVI','NDTI'])
        geemap.ee_export_image(img, filename=f'kursk/{date}.tif', region= geemap.geopandas_to_ee(region_geometry).geometry() ,scale = 10,  file_per_band=False)
        

Generating URL ...
Please wait ...
Data downloaded to c:\Users\User\Google Диск\python_notebooks\total_archangelskiy\field_scale\kursk\2020-10-28.tif
Generating URL ...
Please wait ...
Data downloaded to c:\Users\User\Google Диск\python_notebooks\total_archangelskiy\field_scale\kursk\2020-12-07.tif
Generating URL ...
Please wait ...
Data downloaded to c:\Users\User\Google Диск\python_notebooks\total_archangelskiy\field_scale\kursk\2020-12-10.tif
Generating URL ...
Please wait ...
Data downloaded to c:\Users\User\Google Диск\python_notebooks\total_archangelskiy\field_scale\kursk\2021-02-13.tif
Generating URL ...
Please wait ...
Data downloaded to c:\Users\User\Google Диск\python_notebooks\total_archangelskiy\field_scale\kursk\2021-02-15.tif
Generating URL ...
Please wait ...
Data downloaded to c:\Users\User\Google Диск\python_notebooks\total_archangelskiy\field_scale\kursk\2021-02-18.tif
Generating URL ...
Please wait ...
Data downloaded to c:\Users\User\Google Диск\python_notebooks\tot

In [54]:
for_minNDTI = field_scale("2021-07-15" , finish , ROIs , region_geometry)
for_minNDTI.get_collection()
minNDTI = for_minNDTI.result_collection.select('NDTI').min()
CRS = for_minNDTI.result_collection.first().select('B1').projection().getInfo()['crs']
minNDTI = minNDTI.reproject(CRS)


In [55]:
geemap.ee_export_image(minNDTI, filename='kursk/minNDTI_kursk.tif', region= geemap.geopandas_to_ee(region_geometry).geometry() ,scale = 10,  file_per_band=False)


Generating URL ...
Please wait ...
Data downloaded to c:\Users\User\Google Диск\python_notebooks\total_archangelskiy\field_scale\kursk\minNDTI_kursk.tif


In [27]:
img = for_NDTI.result_collection.first()